In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scanpy as sc

In [ ]:
%matplotlib inline

In [ ]:
import warnings

warnings.filterwarnings('ignore')

In [ ]:
sc.set_figure_params(dpi=80, figsize=(4,4))

sc.settings.verbosity=0

In [ ]:
# Read cellranger files for all four samples
filename = './GSE132188_RAW/E12_5/matrix.mtx'
filename_genes = './GSE132188_RAW/E12_5/genes.tsv'
filename_barcodes = './GSE132188_RAW/E12_5/barcodes.tsv'

#cells should be as rows and genes as columns
e125 = sc.read(filename).transpose()

e125.var_names = np.genfromtxt(filename_genes, dtype=str)[:, 1]
e125.obs_names = np.genfromtxt(filename_barcodes, dtype=str)

In [ ]:
e125

In [ ]:
filename = './GSE132188_RAW/E13_5/matrix.mtx'
filename_genes = './GSE132188_RAW/E13_5/genes.tsv'
filename_barcodes = './GSE132188_RAW/E13_5/barcodes.tsv'

e135 = sc.read(filename).transpose()
e135.var_names = np.genfromtxt(filename_genes, dtype=str)[:, 1]
e135.obs_names = np.genfromtxt(filename_barcodes, dtype=str)

In [ ]:
e135

In [ ]:
filename = './GSE132188_RAW/E14_5/matrix.mtx'
filename_genes = './GSE132188_RAW/E14_5/genes.tsv'
filename_barcodes = './GSE132188_RAW/E14_5/barcodes.tsv'

e145 = sc.read(filename).transpose()
e145.var_names = np.genfromtxt(filename_genes, dtype=str)[:, 1]
e145.obs_names = np.genfromtxt(filename_barcodes, dtype=str)

In [ ]:
e145

In [ ]:
filename = './GSE132188_RAW/E15_5/matrix.mtx'
filename_genes = './GSE132188_RAW/E15_5/genes.tsv'
filename_barcodes = './GSE132188_RAW/E15_5/barcodes.tsv'

e155 = sc.read(filename).transpose()
e155.var_names = np.genfromtxt(filename_genes, dtype=str)[:, 1]
e155.obs_names = np.genfromtxt(filename_barcodes, dtype=str)

In [ ]:
genes=pd.read_csv(filename_genes, header=None, sep="\t")
genes

In [ ]:
e155

In [ ]:
# Add dev. timepoint label for each sample
e125.obs['day'] = '12.5'
e135.obs['day'] = '13.5'
e145.obs['day'] = '14.5'
e155.obs['day'] = '15.5'

In [ ]:
#create a list of four anndata objects
adatas=[e125, e135, e145, e155]

In [ ]:
#making observation and variable names unique for each anndata object
for i, adata in enumerate(adatas):
    # Reset index with a suffix to ensure uniqueness
    adata.obs_names_make_unique(join='-')
    adata.var_names_make_unique(join='-')

In [ ]:
# Create Concatenated anndata object for all timepoints
adata = adatas[0].concatenate(adatas[1:], batch_key='batch', index_unique='-')
# Deleting individual day arrays
#del e125
#del e135
#del e145
#del e155

In [ ]:
adata.obs

In [ ]:
# mitochondrial genes
adata.var["mt"] = adata.var_names.str.startswith("mt-")

In [ ]:
adata

In [ ]:
adata.var['mt'].value_counts()

In [ ]:
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], inplace=True)

In [ ]:
adata

In [ ]:
sc.pl.violin(adata, keys=["n_genes_by_counts"])
sc.pl.violin(adata, keys=["total_counts"])
sc.pl.violin(adata, keys=["pct_counts_mt"])

In [ ]:
sns.displot(adata.obs["total_counts"], bins=100, kde=False)

In [ ]:
sns.displot(adata.obs["log1p_total_counts"], bins=100, kde=False)

In [ ]:
# #counts per cell
adata.obs['n_counts'] = adata.X.sum(1)
# #logcounts per cell
adata.obs['log_counts'] = np.log(adata.obs['n_counts'])
# #genes per cell
adata.obs['n_genes'] = (adata.X > 0).sum(1)
# mitochondrial gene fraction
mt_gene_mask = [gene.startswith('mt-') for gene in adata.var_names]
mt_gene_index = np.where(mt_gene_mask)[0]
adata.obs['mt_frac'] = adata.X[:,mt_gene_index].sum(1) / adata.X.sum(1)

In [ ]:
adata

In [ ]:
adata.raw=adata

In [ ]:
print('Total number of cells: {:d}'.format(adata.n_obs))
adata = adata[adata.obs['mt_frac'] < 0.2]
print('Number of cells after MT filter: {:d}'.format(adata.n_obs))

sc.pp.filter_cells(adata, min_genes = 1200)
print('Number of cells after gene filter: {:d}'.format(adata.n_obs))
#Filter genes:
print('Total number of genes: {:d}'.format(adata.n_vars))

# Min 20 cells - filters out 0 count genes
sc.pp.filter_genes(adata, min_cells=20)
print('Number of genes after cell filter: {:d}'.format(adata.n_vars))

In [ ]:
adata

In [ ]:
adata.raw.to_adata()

In [ ]:
sc.external.pp.scrublet(adata, batch_key="batch")

In [ ]:
adata

In [ ]:
scales_counts = sc.pp.normalize_total(adata, inplace=False)
# log1p transform
adata.layers["log1p_norm"] = sc.pp.log1p(scales_counts["X"], copy=True)

In [ ]:
adata.X.toarray()

In [ ]:
adata.layers["log1p_norm"].toarray()

In [ ]:
sc.pp.highly_variable_genes(adata, layer="log1p_norm", batch_key="batch", n_top_genes=4000)

In [ ]:
adata

In [ ]:
ax = sns.scatterplot(
    data=adata.var, x="means", y="dispersions", hue="highly_variable", s=5
)
ax.set_xlim(None, 1.5)
ax.set_ylim(None, 3)
plt.show()

In [ ]:
adata.write("pan_endo.h5ad")